<a href="https://colab.research.google.com/github/Eran707/MSc-Computational-Neuroscience-Repo/blob/master/Multicompartment_Sim_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Multicompartment Simulator**

**Eran Frank Shorer**

**MSc Computational Neuroscience**

**"Investigating the effects of impermeant anions on the electrical and computational properties of neurons"**




## Instructions:

This jupyter notebook consists of cells. Some cells are plain text, while others are code. 

Press Shift-Enter simultaneously, or click the cell and press the play button in the main toolbar above to run the cell. 

The output of the cell should be displayed below the cell. 



## **Import classes and modules** 


In [1]:
import pandas as pd

try:
    from ipywidgets import widgets, Layout, interact, interactive, interactive_output, Dropdown
    from IPython.display import display
    import bqplot as bqp
    from bqplot import pyplot as plt # bqplot is a plotting for jupyter
    import matplotlib.pyplot as mplt
    import common
    import constants
    import compartment
    import electrodiffusion
    import numpy as np
    import random
    import seaborn as sns 
    import pandas as pd
    import graphing as gr
    #!pip install viola
    ########## Modules required for artist drawing on matplotlib
    import matplotlib.path as mpath
    import matplotlib.lines as mlines
    import matplotlib.patches as mpatches
    from matplotlib.collections import PatchCollection
    
  
    
    
except ModuleNotFoundError:
        print("A class you are trying to import is not present in the directory")
    
except Exception:
        print("Something went wrong - modules have not been imported")

else: 
        print("All relevant classes imported")
    





All relevant classes imported


## A1) Set compartment parameters:

In [2]:
### WIDGET INITIALIZATION ################################################################################
txt_name = widgets.Text(value="Comp1")
lbl_name = widgets.Label(value ="Compartment Name: ")
name_box = widgets.HBox(children = [lbl_name,txt_name])


lbl_rad = widgets.Label(value = "Compartment Radius (um): ")
sldr_rad = widgets.FloatSlider(value=5, min=1, max=10.0, step=0.5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
rad_box = widgets.HBox(children = [lbl_rad,sldr_rad])

lbl_len = widgets.Label(value = "Compartment Length (um): ")
sldr_len = widgets.FloatSlider(value=25, min=0, max=50, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
len_box = widgets.HBox(children = [lbl_len,sldr_len])

lbl_nai = widgets.Label(value = "Na+ concentration (mM): ")
sldr_nai = widgets.FloatSlider(value=14, min=0, max=100, step=10, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_nai = widgets.HBox(children = [lbl_nai,sldr_nai])

lbl_ki = widgets.Label(value = "K+ concentration (mM): ")
sldr_ki = widgets.FloatSlider(value=122, min=0, max=250, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_ki = widgets.HBox(children = [lbl_ki,sldr_ki])

lbl_cli = widgets.Label(value = "Cl- concentration (mM): ")
sldr_cli = widgets.FloatSlider(value=5, min=0, max=50, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_cli = widgets.HBox(children = [lbl_cli,sldr_cli])

lbl_xi = widgets.Label(value = "Impermeant anion concentration (mM): ")
sldr_xi = widgets.FloatSlider(value=154.9, min=0, max=600, step=10, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.2f')
hbox_xi = widgets.HBox(children = [lbl_xi,sldr_xi])

lbl_zi = widgets.Label(value = "Impermeant anion average charge: ")
sldr_zi = widgets.FloatSlider(value=-0.85, min=-3, max=0, step=0.05, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.2f')
hbox_zi = widgets.HBox(children = [lbl_zi,sldr_zi])

lbl_neutral_start = widgets.Label(value = "Adjust starting concentrations for electroneutrality")
chk_neutral_start = widgets.Checkbox(value = True)
hbox_neutral_start = widgets.HBox(children = [lbl_neutral_start,chk_neutral_start])

########################################################################################################


global name, comp_arr, df_sim


#CREATION OF THE PANDAS DATA FRAME THAT WILL KEEP TRACK OF ALL THE VARIABLES IN REAL TIME
df_sim = pd.DataFrame()

comp_arr =[]

output1 = widgets.Output()


btn_new_comp = widgets.Button(description = "Create Compartment",button_style ="success")
btn_defaults = widgets.Button(description = "Reset to defaults")
hbox_btns = widgets.HBox(children = [btn_new_comp, btn_defaults])



def new_comp_clicked(b):
    
    name = txt_name.value
    for j in range (len(comp_arr)):
        if name == comp_arr[j].name:
            txt_name.color = 'red'
            raise Exception("Two compartments cannot have the same name")
    new_comp = compartment.Compartment(name,radius=sldr_rad.value*1e-5, length=sldr_len.value*1e-5,p=-1)
    new_comp.set_ion_properties(na_i=sldr_nai.value*1e-3, k_i = sldr_ki.value*1e-3, cl_i=sldr_cli.value*1e-3,x_i=sldr_xi.value*1e-3, z_i=sldr_zi.value)
    
    if chk_neutral_start.value == True:
        new_comp.osmol_neutral_start()
    
    comp_arr.append(new_comp)
    arr_df = new_comp.get_df_array()
    df_sim[name] = arr_df
    df_sim.index = ['Radius', 'Length', 'Volume', 'Na_i', 'K_i', 'Cl_i', 'X_i', 'z_i', 'ATPase pump rate','KCC2 pump rate','Vm', 'Ek', 'ECl']
    with output1:
        print("Compartment: ("+ new_comp.name +") created" )
        print(new_comp.get_df_array())
      
btn_new_comp.on_click(new_comp_clicked)

def btn_defaults_clicked(b):
    
    
    sldr_rad.value=1
    sldr_len.value = 10
    sldr_nai.value= 14
    sldr_ki.value = 122
    sldr_cli.value = 5
    sldr_xi.value = 150
    sldr_zi.value = -0.85
    
btn_defaults.on_click(btn_defaults_clicked)


vbox_grand = widgets.VBox(children = [name_box,rad_box,len_box, hbox_nai,hbox_ki,hbox_cli,hbox_xi,hbox_zi, hbox_neutral_start, hbox_btns], layout=Layout(border ="solid"))

display(output1,vbox_grand)







Output()

## A2) Review & edit compartments

In [3]:
################################################################################
#  WIDGET INITIALIZATION

btn_review = widgets.Button(description = "Review compartments", button_style ="success")
output2 = widgets.Output()
display(btn_review,output2)

btn_edit = widgets.Button(description = "Edit compartments", button_style ="success")
output3 = widgets.Output()

btn_save_edit = widgets.Button(description = "Save change", button_style ="success",visible='false')


def btn_review_clicked(b):
    output2.clear_output()
    with output2:
        display(df_sim)

btn_review.on_click(btn_review_clicked)       





display(btn_edit,output3)




def btn_edit_clicked(b):
    
    output3.clear_output()
    opts_comp =[]
    for i in range(len(comp_arr)):
        opts_comp.append(comp_arr[i].name)

    global drp_comp,drp_param,txt_new_val
    drp_comp=widgets.Dropdown(options=opts_comp,description='Compartment:',visible='false')
    opts_param=df_sim.index.array[0:10]
    drp_param=widgets.Dropdown(options=opts_param, description='Parameter:', visible='false')

    txt_new_val=widgets.Text(value='', description='New value:', visible='false')
     

    
    drp_param.visible ="true"
    drp_comp.visible ="true"
    txt_new_val.visible="true"
    btn_save_edit.visible ='true'
    
    with output3:
        display(drp_comp,drp_param,txt_new_val,btn_save_edit)
  




btn_edit.on_click(btn_edit_clicked)




def btn_save_edit_clicked(b):
    comp = drp_comp.value
    param = drp_param.value
    change = txt_new_val.value
    
    df_sim.loc[param,comp] = change
    output2.clear_output()
    output3.clear_output()
    with output3:
        display(df_sim)

btn_save_edit.on_click(btn_save_edit_clicked)  



Button(button_style='success', description='Review compartments', style=ButtonStyle())

Output()

Button(button_style='success', description='Edit compartments', style=ButtonStyle())

Output()

## *A3) Simulation settings*

In [4]:
global total_t,dt,t_arr, constant_j_atp


############################################################################
#TIMING

lbl_total_t = widgets.Label(value="Set the simulation run time (minutes):")

sldr_total_t = widgets.FloatSlider(value=1, min=0.5, max=40, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')
HBox_total_t = widgets.HBox(children = [lbl_total_t,sldr_total_t])


lbl_dt = widgets.Label(value="Set the simulation time step (milliseconds):")
sldr_dt = widgets.FloatSlider(value=1, min=1, max=5, step=0.2, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')
HBox_dt = widgets.HBox(children = [lbl_dt,sldr_dt])

############################################################################
# ATPase PROPERTIES

lbl_atpase = widgets.Label(value = "ATPase model type:")
rdb_atpase = widgets.RadioButtons(options=['J_ATP = p * (Na_in/Na_out)^3', 'Constant'])

hbox_atpase = widgets.HBox(children = [lbl_atpase,rdb_atpase], layout =Layout(border='solid'))


############################################################################
# EXTERNAL ION PROPERTIES

lbl_ext_ions = widgets.Label(value = "External Ion Parameters:")

lbl_nao = widgets.Label(value = "Na+ concentration (mM): ")
sldr_nao = widgets.FloatSlider(value=145, min=0, max=200, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.2f')
hbox_nao = widgets.HBox(children = [lbl_nao,sldr_nao])

lbl_ko = widgets.Label(value = "K+ concentration (mM): ")
sldr_ko = widgets.FloatSlider(value=3.5, min=0, max=10, step=0.5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_ko = widgets.HBox(children = [lbl_ko,sldr_ko])

lbl_clo = widgets.Label(value = "Cl- concentration (mM): ")
sldr_clo = widgets.FloatSlider(value=119, min=100, max=200, step=1, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.2f')
hbox_clo = widgets.HBox(children = [lbl_clo,sldr_clo])

lbl_xo = widgets.Label(value = "Impermeant anion concentration (mM): ")
sldr_xo = widgets.FloatSlider(value=29.5, min=0, max=50, step=0.5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.2f')
hbox_xo = widgets.HBox(children = [lbl_xo,sldr_xo])

lbl_zo = widgets.Label(value = "Impermeant anion average charge: ")
sldr_zo = widgets.FloatSlider(value=-0.85, min=-3, max=0, step=0.05, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.2f')
hbox_zo = widgets.HBox(children = [lbl_zo,sldr_zo])

vbox_ext_ions = widgets.VBox(children = [lbl_ext_ions,hbox_nao,hbox_ko,hbox_clo,hbox_xo,hbox_zo],layout =Layout(border='solid'))

t_arr = [0]


vbox_t = widgets.VBox([HBox_total_t,HBox_dt], layout=Layout(border='solid'))



display(vbox_t,hbox_atpase, vbox_ext_ions)




## A4) Anion Flux Settings:


### i ) Increase intracellular impermeant anions concentration during simulation

Changing the concentration and charge of impermeant anions replicates changes such a increased protein transcription.

In [5]:
global xflux_start_t, xflux_end_t, xflux_on
###############################################################################
# Widget Initialization
# Simulating protein synthesis

output_xflux = widgets.Output()

lbl_xflux = widgets.Label(value="Add intracellular impermeant anions mid simulation:")

chk_xflux = widgets.Checkbox(value = False)
hbox_xflux = widgets.HBox(children = [lbl_xflux, chk_xflux])
vbox_xflux_1 = widgets.VBox(children = [hbox_xflux])

xflux_on = False

with output_xflux:
    display(vbox_xflux_1)
    
lbl_xflux_comp = widgets.Label (value = "Select compartment")
drp_xflux_comp = widgets.Dropdown()
comp_options = []
comp_options.append("All compartments")
for i in range(len(comp_arr)):
    comp_options.append(comp_arr[i].name)
drp_xflux_comp.options = comp_options
hbox_xflux_comp = widgets.HBox(children = [lbl_xflux_comp, drp_xflux_comp])

lbl_xflux_conc = widgets.Label(value="Concentration of anions to add (mM)")
sldr_xflux_conc = widgets.FloatSlider(value=2, min=0, max=10, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='.2f')
hbox_xflux_conc = widgets.HBox(children = [lbl_xflux_conc,sldr_xflux_conc])

lbl_xflux_charge = widgets.Label(value="Charge of added anions")
sldr_xflux_charge = widgets.FloatSlider(value=-0.85, min=-2, max=-0.1, step=0.05, disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='.2f')
hbox_xflux_charge = widgets.HBox(children = [lbl_xflux_charge,sldr_xflux_charge])

lbl_xflux_start_t = widgets.Label(value="Time to start anion flux (s)")
sldr_xflux_start_t = widgets.FloatSlider(value=1, min=0.5, max=10000, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='.1f')
hbox_xflux_start_t = widgets.HBox(children = [lbl_xflux_start_t,sldr_xflux_start_t])

lbl_xflux_end_t = widgets.Label(value="Time to end anion flux (s)")
sldr_xflux_end_t = widgets.FloatSlider(value=1, min=0.5, max=10000, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',       readout=True, readout_format='.1f')
hbox_xflux_end_t = widgets.HBox(children = [lbl_xflux_end_t,sldr_xflux_end_t])

vbox_xflux_2 = widgets.VBox(children =[hbox_xflux_comp, hbox_xflux_conc,  hbox_xflux_charge, hbox_xflux_start_t, hbox_xflux_end_t ], layout=Layout(border ="solid"))
#############################################################################

display(output_xflux)


def set_x_flux(b):
    
    if chk_xflux.value == True:
        
        xflux_on = True
        
        
        sldr_xflux_start_t.value = sldr_total_t.value *60 /5
        sldr_xflux_start_t.max = sldr_total_t.value *60
        sldr_xflux_end_t.value = sldr_total_t.value *60
        sldr_xflux_end_t.max = sldr_total_t.value *60
        
          
        output_xflux.clear_output()    
        with output_xflux:
            display(vbox_xflux_1, vbox_xflux_2)
        
        
    elif chk_xflux.value == False:
        xflux_on = False
        output_xflux.clear_output()
        display(vbox_xflux_1)
        
        
chk_xflux.observe(set_x_flux)
 

Output()

### ii) Alter charge of intracellular impermeant anions

Changing the charge of impermeant anions (without changing concentrations)  replicates post-translational protein modifications that alters protein charge, e.g. phosphorylation, protonation of impermeant anions.


In [6]:

lbl_zflux = widgets.Label(value="Change intracellular impermeant anions charge during simulation:")
lbl_zflux_2 = widgets.Label(value="______Modelling change to average charge of impermeants ______")
chk_zflux = widgets.Checkbox(value = False)
hbox_zflux = widgets.HBox(children = [lbl_zflux, chk_zflux])
vbox_zflux_1 = widgets.VBox(children = [lbl_zflux_2,hbox_zflux])


output_zflux = widgets.Output()
with output_zflux:
    display(vbox_zflux_1)


    
zflux_on = False

lbl_zflux_comp = widgets.Label (value = "Select compartment")
drp_zflux_comp = widgets.Dropdown()
comp_options = []
comp_options.append("All compartments")
for i in range(len(comp_arr)):
    comp_options.append(comp_arr[i].name)
drp_zflux_comp.options = comp_options
hbox_zflux_comp = widgets.HBox(children = [lbl_zflux_comp, drp_zflux_comp])
    
lbl_zflux_charge = widgets.Label(value="Desired average charge of impermeant anions")
sldr_zflux_charge = widgets.FloatSlider(value=-0.85, min=-3, max= 1, step=0.05, disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='.2f')
hbox_zflux_charge = widgets.HBox(children = [lbl_zflux_charge,sldr_zflux_charge])

lbl_zflux_start_t = widgets.Label(value="Time to start anion flux (s)")
sldr_zflux_start_t = widgets.FloatSlider(value=1, min=0.5, max=10000, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='.1f')
hbox_zflux_start_t = widgets.HBox(children = [lbl_zflux_start_t,sldr_zflux_start_t])

lbl_zflux_end_t = widgets.Label(value="Time to end anion flux (s)")
sldr_zflux_end_t = widgets.FloatSlider(value=1, min=0.5, max=10000, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',       readout=True, readout_format='.1f')
hbox_zflux_end_t = widgets.HBox(children = [lbl_zflux_end_t,sldr_zflux_end_t])

vbox_zflux_2 = widgets.VBox(children =[hbox_zflux_comp,hbox_zflux_charge, hbox_zflux_start_t, hbox_zflux_end_t ], layout=Layout(border ="solid"))



display(output_zflux)

def set_z_flux(b):
    
    if chk_zflux.value == True:
        
        zflux_on = True
        
        
        sldr_zflux_start_t.value = sldr_total_t.value *60 /5
        sldr_zflux_start_t.max = sldr_total_t.value *60
        sldr_zflux_end_t.value = sldr_total_t.value *60
        sldr_zflux_end_t.max = sldr_total_t.value *60
        
          
        output_zflux.clear_output()    
        with output_zflux:
            display(vbox_zflux_1, vbox_zflux_2)
        #print(x_flux_on)
        
    elif chk_zflux.value == False:
        zflux_on = False
        output_zflux.clear_output()
        display(vbox_zflux_1)
        
        
        
    

chk_zflux.observe(set_z_flux)


Output()

### iii) Increase extracellular impermeant anions


In [7]:
output_xoflux = widgets.Output()

lbl_xoflux = widgets.Label(value="Change extracellular impermeant anions mid simulation:")

chk_xoflux = widgets.Checkbox(value = False)
hbox_xoflux = widgets.HBox(children = [lbl_xoflux, chk_xoflux])
vbox_xoflux_1 = widgets.VBox(children = [hbox_xoflux])

xoflux_on = False

with output_xoflux:
    display(vbox_xoflux_1)
    
lbl_xoflux_conc = widgets.Label(value="Concentration of extracellular impermeant anions to add/subtract (mM)")
sldr_xoflux_conc = widgets.FloatSlider(value=30, min=-60, max=60, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='.2f')
hbox_xoflux_conc = widgets.HBox(children = [lbl_xoflux_conc,sldr_xoflux_conc])

lbl_xoflux_charge = widgets.Label(value="Charge of extracellular impermeant anions")
sldr_xoflux_charge = widgets.FloatSlider(value=-0.85, min=-2, max=-0.1, step=0.05, disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='.2f')
hbox_xoflux_charge = widgets.HBox(children = [lbl_xoflux_charge,sldr_xoflux_charge])

lbl_xoflux_start_t = widgets.Label(value="Time to start extracellular anion flux (s)")
sldr_xoflux_start_t = widgets.FloatSlider(value=1, min=0.5, max=10000, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='.1f')
hbox_xoflux_start_t = widgets.HBox(children = [lbl_xoflux_start_t,sldr_xoflux_start_t])

lbl_xoflux_end_t = widgets.Label(value="Time to end extracellular anion flux (s)")
sldr_xoflux_end_t = widgets.FloatSlider(value=1, min=0.5, max=10000, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',       readout=True, readout_format='.1f')
hbox_xoflux_end_t = widgets.HBox(children = [lbl_xoflux_end_t,sldr_xoflux_end_t])

vbox_xoflux_2 = widgets.VBox(children =[hbox_xoflux_conc,  hbox_xoflux_charge, hbox_xoflux_start_t, hbox_xoflux_end_t ], layout=Layout(border ="solid"))
#############################################################################

display(output_xoflux)


def set_xo_flux(b):
    
    if chk_xoflux.value == True:
        
        xoflux_on = True
        
        
        sldr_xoflux_start_t.value = sldr_total_t.value *60 /5
        sldr_xoflux_start_t.max = sldr_total_t.value *60
        sldr_xoflux_end_t.value = sldr_total_t.value *60
        sldr_xoflux_end_t.max = sldr_total_t.value *60
        
          
        output_xoflux.clear_output()    
        with output_xoflux:
            display(vbox_xoflux_1, vbox_xoflux_2)
        
        
    elif chk_xoflux.value == False:
        xoflux_on = False
        output_xoflux.clear_output()
        display(vbox_xoflux_1)
        
        
chk_xoflux.observe(set_xo_flux)
 

Output()

## A5) Add synaptic input


In [8]:
#########################################################
# Widget initialization


btn_synapse = widgets.Button(description = 'Add Synapse', button_style='info')
btn_synapse_confirm = widgets.Button(description = 'Confirm synapse', buttons_style ='success')
btn_synapse_reset =widgets.Button(description ='Reset all synapses', button_style = 'danger',disabled =True)

output_synapse =widgets.Output()
output_synapse_list =widgets.Output()
vbox_synapse = widgets.VBox(children=[output_synapse,output_synapse_list])

lbl_synapse_comp = widgets.Label(value ="Select Compartment: ")
drp_synapse_comp = widgets.Dropdown()
drp_synapse_comp_options = []
for i in range(len(comp_arr)):
    drp_synapse_comp_options.append(comp_arr[i].name)
drp_synapse_comp.options = drp_synapse_comp_options
hbox_synapse_comp = widgets.HBox(children=[lbl_synapse_comp, drp_synapse_comp])



rdb_synapse_type = widgets.RadioButtons(options = ["Excitatory (Glutamatergic)","Inhibitory (Gabaergic)"])
lbl_synapse_current =widgets.Label(value ='Current (mA): ')
sldr_synapse_current = widgets.FloatSlider( min = 0, value=10, max=20)
hbox_synapse_current = widgets.HBox(children=[lbl_synapse_current,sldr_synapse_current])

total_t = sldr_total_t.value
dt = sldr_dt.value

lbl_synapse_t_start = widgets.Label(value="Start time(min)")
sldr_synapse_t_start = widgets.FloatSlider( min = 0, value= 10, max= total_t) 
hbox_synapse_t_start = widgets.HBox(children = [lbl_synapse_t_start, sldr_synapse_t_start])

lbl_synapse_t_end = widgets.Label(value="End time(min)")
sldr_synapse_t_end = widgets.FloatSlider( min = 0, value= 10, max= total_t) 
hbox_synapse_t_end = widgets.HBox(children = [lbl_synapse_t_end, sldr_synapse_t_end])

##################
    
#display(output_synapse)  

display(btn_synapse)
global synapse_number
global synapse_arr
 
synapse_number = 0
synapse_arr = [] # 2D array of all the synapses that have been added

def btn_synapse_clicked(b):
    with output_synapse:
        display(hbox_synapse_comp, rdb_synapse_type, hbox_synapse_current, hbox_synapse_t_start, hbox_synapse_t_end, btn_synapse_confirm,btn_synapse_reset)
    btn_synapse.disabled = True
    output_synapse.layout = Layout(border='solid')
    

    
def btn_synapse_confirm_clicked(b):
       
    syn_num = len(synapse_arr) 
    syn_comp = drp_synapse_comp.value
    syn_type = rdb_synapse_type.value
    syn_current = sldr_synapse_current.value
    syn_t_start = sldr_synapse_t_start.value
    syn_t_end = sldr_synapse_t_end.value
    synapse_arr.append([syn_num,syn_comp,syn_type, syn_current, syn_t_start, syn_t_end ])
    
    
    
   
    with output_synapse_list:
           print(synapse_arr[-1])

    
    
    
    btn_synapse_reset.disabled=False
        
        
def btn_synapse_reset_clicked(b):
    output_synapse_list.clear_output()
    synapse_arr.clear()

btn_synapse.on_click(btn_synapse_clicked)
btn_synapse_confirm.on_click(btn_synapse_confirm_clicked)
btn_synapse_reset.on_click(btn_synapse_reset_clicked)

display(vbox_synapse)




Button(button_style='info', description='Add Synapse', style=ButtonStyle())

## A6) Pre-simulation summary

In [9]:
###################################################################
# WIDGET INITIALIZATION

btn_summary = widgets.Button(description = 'View Presimulation summary ',button_style = "info", layout=Layout(width='20%'))
output_summary = widgets.Output()
display(btn_summary, output_summary)





def btn_summary_clicked(b):
    with output_summary:
        print("*********************")
        print("Compartment settings:")
        print("*********************")
        print(df_sim)
        print()
        print("*********************")
        print("Timing:")
        print("*********************")
        print("Total time (mins): " + str(total_t))
        print("Timestep (ms): " + str(dt))
        print()
        print("*********************")
        print("Extracellular anion concentrations:")
        print("*********************")
        print("Na: " + str(sldr_nao.value) + " mM")
        print("K: " + str(sldr_ko.value) + " mM")
        print("Cl: " + str(sldr_clo.value) + " mM")
        print("X: " + str(sldr_xo.value) + " mM")
        print()
        print("*********************")
        print("ATPase settings:")
        print("*********************")
        print("Model type: " + rdb_atpase.value)
        print("Pump rate: "  )
        print()
        print("*********************")
        print("Impermeant anion changes:")
        print("*********************")
        
        if chk_xflux.value == True:
            print(drp_xflux_comp.value+" : increase intracellular impermeant anion concentration -  "+ str(sldr_xflux_conc.value)+" mM, valence: " + str(sldr_xflux_charge.value) +
                 ", between: " + str(sldr_xflux_start_t.value) + "s and  "+  str(sldr_xflux_end_t.value) + "s")
        else:
            print("No change of intracellular impermeant anion concentration mid simulation")
            
        if chk_zflux.value == True:
            print(drp_zflux_comp.value+" : change in intracellular impermeant anion charge - "+  " valence: " + str(sldr_zflux_charge.value) +
                 ", between: " + str(sldr_zflux_start_t.value) + "s and  "+  str(sldr_zflux_end_t.value) + "s")
            
        else:
            print("No change of intracellular  impermeant anion charge mid simulation")
            
        if chk_xoflux.value == True:
            print("Change in extracellular impermeant anions - " + str(sldr_xoflux_conc.value)+ " mM, valence: " + str(sldr_xoflux_charge.value) +
                 ", between: " + str(sldr_xoflux_start_t.value) + "s and  "+  str(sldr_xoflux_end_t.value) + "s")
            
        else:
            print("No change of extracellular impermeant anion concentration mid simulation")

btn_summary.on_click(btn_summary_clicked)




Button(button_style='info', description='View Presimulation summary ', layout=Layout(width='20%'), style=Butto…

Output()

# *B) Run Simulation:*

In [10]:

btn_sim = widgets.Button(description ="Run Simulation", button_style="Danger",disabled=False)
output5 = widgets.Output()
output6 = widgets.Output()
output7 = widgets.Output()



display(output6, output7, btn_sim)

global SIM_OVER

SIM_OVER = False

def btn_sim_clicked(b):
  
    
    #########
    # ELECTRODIFFUSION LINKAGES
    
    global ed_arr
    ed_arr = [] # array of all the electro-diffusion links between the compartments
    ed_dict_arr = [] # array of all the electro-diffusion dictionaries (constantly changing)
    ed_conc_changes_arr = []
    for e in range(len(comp_arr)-1):
        ed_arr.append(electrodiffusion.Electrodiffusion(comp_arr[e],comp_arr[e+1]))
        
    ED_ON = True
    
    ##########
    ##########
    # TIMING 
    
    global dt,total_t
    dt=sldr_dt.value*1e-3
    total_t= sldr_total_t.value *60
    interval = total_t/dt
    run_t=0
  
    prg = widgets.FloatProgress(description='Simulating...', min=0,step=dt,max=total_t,value=0,continuous_update=True)
    lbl_prg = widgets.Label(value = "Percent complete:" + str(round(run_t/total_t*100,2) ))
    with output6:
        display(prg, lbl_prg)
    
    #########
    #########
    # ATPASE SETTINGS
    
    if rdb_atpase.value == 'Constant':
        j_atpase_constant = True
    else: j_atpase_constant = False
    
    ########
    # EXTERNAL ION PARAMETERS:
    nao = sldr_nao.value*1e-3
    ko = sldr_ko.value*1e-3
    clo = sldr_clo.value*1e-3
    xo = sldr_xo.value*1e-3
    zo = sldr_zo.value
    
    for p in range(len(comp_arr)):
        comp_arr[p].set_external_ion_properties(nao,ko,clo,xo,zo)
        
    
    ########
    # X-FLUX & Z-FLUX SETTINGS
    
    xflux_on = chk_xflux.value
    
    
    
    if xflux_on == True:
        
        for i in range(len(comp_arr)):
            if drp_xflux_comp.value == comp_arr[i].name or drp_xflux_comp.value == "All compartments":
                comp_arr[i].xflux_switch = True
        
        xflux_start_t = sldr_xflux_start_t.value
        xflux_end_t = sldr_xflux_end_t.value
        xflux_conc = sldr_xflux_conc.value*1e-3
        xflux_charge = sldr_xflux_charge.value

    zflux_on = chk_zflux.value
    
    if zflux_on == True: 
        
        for i in range(len(comp_arr)):
            if drp_zflux_comp.value == comp_arr[i].name or drp_zflux_comp.value == "All compartments":
                comp_arr[i].zflux_switch = True
            
        zflux_start_t = sldr_zflux_start_t.value
        zflux_end_t = sldr_zflux_end_t.value
        zflux_charge = sldr_zflux_charge.value
        
    xoflux_on = chk_xoflux.value
    
    if xoflux_on == True:
        
        xoflux_start_t = sldr_xoflux_start_t.value
        xoflux_end_t = sldr_xoflux_end_t.value
        xoflux_conc = sldr_xoflux_conc.value*1e-3
        xoflux_charge = sldr_xoflux_charge.value
    
    ########
    ########
    # SIMULATION
    
    while run_t < total_t:

        
        if ED_ON:

            ed_dict_arr = []  # array of all the electro-diffusion dictionaries (constantly changing)
            ed_conc_changes_arr = []
            
            


            for a in range(len(comp_arr)):
                comp_arr[a].step(dt, total_t,run_t, constant_j_atp = j_atpase_constant)  # step for each compartment
                if xflux_on: comp_arr[a].x_flux(run_t,xflux_start_t, xflux_end_t, xflux_conc, xflux_charge)
                if zflux_on: comp_arr[a].z_flux(zflux_start_t, zflux_end_t,zflux_charge)
                if xoflux_on: comp_arr[a].external_xflux(run_t,xoflux_start_t, xoflux_end_t, xoflux_conc, xoflux_charge)
                ed_dict_arr.append(comp_arr[a].get_ed_dict())  # electrodiffusion dictionary for each compartment
                
                

            for b in range(len(comp_arr) - 1):
                ed_conc_changes_arr.append(
                    ed_arr[b].calc_ed(dt, ed_dict_arr[b], ed_dict_arr[b + 1]))  # makes an array of all the ED conc changes

            for c in range(len(comp_arr) - 1):
                comp_arr[c].ed_update(ed_conc_changes_arr[c],
                                      "positive")  # appending the electrodiffusion concentrations for each compartment
                comp_arr[c + 1].ed_update(ed_conc_changes_arr[c], "negative")

            for d in range(len(comp_arr)):
                comp_arr[d].update_volumes()  # updates of the volumes, arrays, and dataframe for each compartment
                if run_t != 0:
                    comp_arr[d].update_arrays()
                df_sim[comp_arr[d].name] = comp_arr[d].get_df_array()
                
            
            if run_t != 0:
                t_arr.append(run_t)
                
            run_t += dt
            prg.value += dt  
            lbl_prg.value =  "Percent complete:" + str(round(run_t/total_t*100,2)) + '%'

        else: # if you want to run with normal diffusion not ED
            for a in range(len(comp_arr)):
                comp_arr[a].step(dt, total_t,run_t, constant_j_atp = j_atpase_constant)
                comp_arr[a].x_flux(xflux_start_t, xflux_end_t, xflux_conc, xflux_charge)
                comp_arr[a].update_volumes()  # updates of the volumes, arrays, and dataframe for each compartment
                comp_arr[a].update_arrays()
                df_sim[comp_arr[a].name] = comp_arr[d].get_df_array()
    
       
    SIM_OVER = True
    btn_sim.disabled=True
    lbl_finalvals = widgets.Label(value="Final Values:")
    with output7:  
        print("Simulation complete!")
        display(lbl_finalvals,df_sim)
        
        



        
btn_sim.on_click(btn_sim_clicked)  



Output()

Output()

Button(button_style='danger', description='Run Simulation', style=ButtonStyle())

## C) Graphing


In [11]:

SIM_OVER == True ## * Just for testing purposes

#graphing module can onlt be accessed once the simulation is complete

    ##### User interface:

drp_comp = widgets.Dropdown(description='Compartment:')

options_drp_comp = []
options_drp_comp.append('ALL COMPARTMENTS')

for i in range(len(comp_arr)):
    options_drp_comp.append(comp_arr[i].name)

drp_comp.options = options_drp_comp    


drp_param = widgets.Dropdown(description='Parameter:')
options_drp_param = ['Membrane Potential (Vm)','Cl- Reversal Potential (E-Cl)', 'K+ Reversal Potential (E-K)','Cl- Driving Force'
                        ,'Volume (pL)','',
                         'All ion concentrations','Na+ Conc.','K+ Conc.','Cl- Conc.','Impermeant anion Conc.']


drp_param.options = options_drp_param

intervals = len(t_arr) #amount of time points simulated

sldr_start = widgets.FloatSlider(description='Start time (s)',min=0,step=dt,max=total_t, value=20)
sldr_bins = widgets.FloatSlider(description='Bins',min=50,step=50,max=intervals, value=1000)




hbox_graph = widgets.HBox()
vbox_graph1 = widgets.VBox()
vbox_graph2 = widgets.VBox()
vbox_graph1.children =[drp_comp,drp_param]
vbox_graph2.children =[sldr_start,sldr_bins]
hbox_graph.children = [vbox_graph1,vbox_graph2]

btn_main_graph = widgets.Button(description='Graph it!',button_style='success')
output10 = widgets.Output()
display(hbox_graph,btn_main_graph,output10)

number_of_colors = len(comp_arr)

color_arr = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]

#with output10:
    #print(df_sim) 


def plot_main_graph(param='',comp_num=0,all_comps =False,jump=1):
    
        
        
        
       
        
        
        start_t = int(sldr_start.value * 1000)
        x_arr = t_arr 
    
        bins = sldr_bins.value #bins required
        jump = round(intervals/bins)
        
        x_sc = bqp.LinearScale()
        y_sc = bqp.LinearScale()
        col_sc = bqp.ColorScale(scheme="RdYlGn")
        ax_x = bqp.Axis(scale = x_sc, label="Time(s)", min= start_t, max=t_arr[-1])
        ax_y = bqp.Axis(scale = y_sc, label=param, orientation='vertical', side='left')
        
        
        
        if all_comps == True:
                      
            m_arr = []
            output10.clear_output()
                   
         
            
            for i in range(len(comp_arr)):
                
                col = color_arr[i]
                
                f1_title = "All Compartments"+" : " + "Time (s) vs " + drp_param.value 
                
                if param == 'Membrane Potential (Vm)':
                    m_arr.append(bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[i].v_arr[start_t:-1:jump],colors = [col], 
                       display_legend=True,labels= ['Vm - '+comp_arr[i].name], scales = {'x': x_sc, "y": y_sc}))
                    
                elif param == 'Cl- Reversal Potential (E-Cl)':
                    m_arr.append(bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[i].E_cl_arr[start_t:-1:jump], colors = [col],  
                       display_legend=True,labels= ['E-Cl - '+comp_arr[i].name], scales = {'x': x_sc, "y": y_sc}))
                    
                elif param == 'K+ Reversal Potential (E-K)':
                    m_arr.append(bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[i].E_k_arr[start_t:-1:jump], colors = [col],  
                       display_legend=True,labels= ['E-K - '+comp_arr[i].name], scales = {'x': x_sc, "y": y_sc}))
                    
                elif param == 'Cl- Driving Force':
                    m_arr.append(bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[i].drivingf_cl_arr[start_t:-1:jump], colors = [col],  
                       display_legend=True,labels= ['Cl- Driv.Force '+comp_arr[i].name], scales = {'x': x_sc, "y": y_sc}))
                    
                    
                elif param == 'Volume (pL)':
                    m_arr.append(bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[i].w_arr[start_t:-1:jump], colors = [col],  
                       display_legend=True,labels= ['Vol. - '+comp_arr[i].name ], scales = {'x': x_sc, "y": y_sc}))
                    
                elif param == 'Na+ Conc.':
                    m_arr.append(bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[i].na_arr[start_t:-1:jump], colors = [col],  
                       display_legend=True,labels= ['Na+ - '+comp_arr[i].name], scales = {'x': x_sc, "y": y_sc}))
                    
                elif param == 'K+ Conc.':
                    m_arr.append(bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[i].k_arr[start_t:-1:jump],colors = [col],  
                       display_legend=True,labels= ['K+ - '+comp_arr[i].name], scales = {'x': x_sc, "y": y_sc}))
                    
                elif param == 'Cl- Conc.':
                    m_arr.append(bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[i].cl_arr[start_t:-1:jump],colors = [col],  
                       display_legend=True,labels= ['Cl- - '+comp_arr[i].name], scales = {'x': x_sc, "y": y_sc}))
                    
                elif param == 'Impermeant anion Conc.':
                    m_arr.append(bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[i].x_arr[start_t:-1:jump], colors = [col], 
                       display_legend=True,labels= ['X -'+comp_arr[i].name], scales = {'x': x_sc, "y": y_sc}))        
            
                
                
                       
                
            f1 = bqp.Figure(title = f1_title, marks=m_arr, axes=[ax_x, ax_y],legend_location='top-right',scales={'x': x_sc, 'color': col_sc, 'y': y_sc})
    
            f1.layout.height = '600px'
            f1.layout.width = '800px'
            output10.clear_output()
            with output10:    
                display(f1)
        
        
        if all_comps == False:            
            
            f1_title = comp_arr[comp_num].name+" : " + "Time (s) vs " + drp_param.value 

            if param == 'Membrane Potential (Vm)':
                
                m1 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].v_arr[start_t:-1:jump],colors = ['#000000'], 
                       display_legend=True,labels= ['Vm'], scales = {'x': x_sc, "y": y_sc})
                
            elif param == 'Cl- Reversal Potential (E-Cl)':
                
                m1 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].E_cl_arr[start_t:-1:jump],colors = ['#000000'], 
                       display_legend=True,labels= ['E-Cl'], scales = {'x': x_sc, "y": y_sc})
                
            elif param == 'K+ Reversal Potential (E-K)':
                
                m1 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].E_k_arr[start_t:-1:jump],colors =['#000000'], 
                       display_legend=True,labels= ['E-K'], scales = {'x': x_sc, "y": y_sc})
                
            elif param == 'Cl- Driving Force':
                
                m1 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].drivingf_cl_arr[start_t:-1:jump], colors = ['#000000'],  
                       display_legend=True,labels= ['Cl- Driv.Force '+comp_arr[comp_num].name], scales = {'x': x_sc, "y": y_sc})
                 
            elif param == 'Volume (pL)':
                
                m1 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].w_arr[start_t:-1:jump],colors = ['#000000'], 
                       display_legend=True,labels= ['Vol.'], scales = {'x': x_sc, "y": y_sc})
                
            elif param == 'Na+ Conc.':
                
                m1 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].na_arr[start_t:-1:jump],colors = ['#e7298a'], 
                       display_legend=True,labels= ['Na+'], scales = {'x': x_sc, "y": y_sc})
                
            elif param == 'K+ Conc.':
                
                m1 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].k_arr[start_t:-1:jump],colors = ['#7570b3'], 
                       display_legend=True,labels= ['K+'], scales = {'x': x_sc, "y": y_sc}) 
             
            elif param == 'Cl- Conc.':
                
                m1 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].cl_arr[start_t:-1:jump],colors = ['#1b9e77'], 
                       display_legend=True,labels= ['Cl-'], scales = {'x': x_sc, "y": y_sc}) 
                
            elif param == 'Impermeant anion Conc.':
                
                m1 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].x_arr[start_t:-1:jump],colors = ['#000000'], 
                       display_legend=True,labels= ['X'], scales = {'x': x_sc, "y": y_sc}) 
                
            elif param == 'All ion concentrations':
                
                m1 = []
                
                a1 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].na_arr[start_t:-1:jump],colors = ['#e7298a'], 
                       display_legend=True,labels= ['Na+'], scales = {'x': x_sc, "y": y_sc})
                
                a2 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].k_arr[start_t:-1:jump],colors = ['#7570b3'], 
                       display_legend=True,labels= ['K+'], scales = {'x': x_sc, "y": y_sc}) 
                
                
                a3 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].cl_arr[start_t:-1:jump],colors = ['#1b9e77'], 
                       display_legend=True,labels= ['Cl-'], scales = {'x': x_sc, "y": y_sc})
                
                a4 = bqp.Lines(x= t_arr[start_t:-1:jump],y= comp_arr[comp_num].x_arr[start_t:-1:jump],colors = ['#000000'], 
                       display_legend=True,labels= ['X'], scales = {'x': x_sc, "y": y_sc}) 
                
                m1 = [a1,a2,a3,a4]
                
                
                f1_title = comp_arr[comp_num].name + ": all ion concentrations vs time"
                f1 = bqp.Figure(title = f1_title, marks=m1, axes=[ax_x, ax_y],legend_location='top-right')
                f1.layout.height = '600px'
                f1.layout.width = '800px'
                output10.clear_output()
                with output10:    
                    display(f1)
                return
            
    
    
            f1 = bqp.Figure(title = f1_title, marks=[m1], axes=[ax_x, ax_y],legend_location='top-right')
    
            f1.layout.height = '600px'
            f1.layout.width = '800px'
            output10.clear_output()
            with output10:    
                display(f1)

            
            
        
                
                
            

        #####


def btn_main_graph_clicked(b):
    
    
    start_t = int(sldr_start.value * 1000)
    x_arr = t_arr 
    
    bins = sldr_bins.value #bins required

    jump = round(intervals/bins)
    
    
    
    if drp_comp.value == 'ALL COMPARTMENTS':
    
            param = drp_param.value
            plot_main_graph(param,all_comps=True,jump=jump)

    
    elif drp_comp.value != 'ALL COMPARTMENTS':
        
        
        for i in range(len(comp_arr)):
            if (drp_comp.value == comp_arr[i].name):
                comp_num = i

        param = drp_param.value
        plot_main_graph(param,comp_num,all_comps=False,jump=jump)

    
    
    #if drp_comp != 'ALL COMPARTMENTS':
           
        
btn_main_graph.on_click(btn_main_graph_clicked)

#print(y_arr)

        
        


Button(button_style='success', description='Graph it!', style=ButtonStyle())

Output()

In [12]:


m_flux = bqp.Lines(x= t_arr,y= comp_arr[0].xo_arr,colors = ['#000000'], 
                       display_legend=True,labels= ['X'], scales = {'x': bqp.LinearScale(), "y": bqp.LinearScale()}) 

ax_x = bqp.Axis(scale = bqp.LinearScale(), label="Time(s)")
ax_y = bqp.Axis(scale = bqp.LinearScale(), label="Flux conc", orientation='vertical', side='left')

f = bqp.Figure(title = '', marks=[m_flux], axes=[ax_x, ax_y],legend_location='top-right')

display(f)


#for i in range(len(comp_arr[0].xflux_arr)):
    #if comp_arr[0].xflux_arr[i] > 0:
        #print(comp_arr[0].xflux_arr[i])
        
print(xoflux_on)


Figure(axes=[Axis(label='Time(s)', scale=LinearScale()), Axis(label='Flux conc', orientation='vertical', scale…

False


### Ionic Heatmap

In [13]:
### HEATMAP OF ION CHANGES DURING SIMULATION:

########################
# Widget initialization

btn_heat = widgets.Button(description = 'Show Heat Map')
drp_ion = widgets.Dropdown(description = 'Ion:',options=['Na','K','Cl','X'])
sldr_heat = widgets.IntSlider(description= 'Bins', value= 20, min = 5, max= 100, step=2)
output_heat = widgets.Output()

###########################
# Plotting function on click

def btn_heat_clicked(b):

    y_arr = []
    x_arr = []
    data_arr = []
    bins = sldr_heat.value
    ion = drp_ion.value
    jump = int(total_t*1000/bins)
    
    
    for i in range(len(comp_arr)):
        
        y_arr.append(comp_arr[i].name)
        temp_arr =[]
        
        for j in range(bins):
            time_point = j*jump
            if i==0:
                x_arr.append(round(t_arr[time_point],2))
            if ion == 'Na':
                temp_arr.append(comp_arr[i].na_arr[time_point])
            elif ion == 'K':
                temp_arr.append(comp_arr[i].k_arr[time_point])
            elif ion == 'Cl':
                temp_arr.append(comp_arr[i].cl_arr[time_point])
            elif ion == 'X':
                temp_arr.append(comp_arr[i].x_arr[time_point])
        
        data_arr.append(temp_arr)
                
    
    
    x_sc = bqp.LinearScale()
    y_sc = bqp.LinearScale()
    
    
    
    with output_heat:
        output_heat.clear_output()
        mplt.figure()
        plt_heat = sns.heatmap(data_arr,cmap="BuPu", linewidth=0.5, xticklabels=x_arr, yticklabels=y_arr, cbar_kws={'label': 'Concentration (mM)'})
        mplt.xlabel("Simulation time (s)")
        mplt.ylabel("Compartment")
        mplt.show()
        
    
    #display(output_heat)
    
    
##################################

btn_heat.on_click(btn_heat_clicked)


display(drp_ion, sldr_heat,btn_heat,output_heat)

Dropdown(description='Ion:', options=('Na', 'K', 'Cl', 'X'), value='Na')

IntSlider(value=20, description='Bins', min=5, step=2)

Button(description='Show Heat Map', style=ButtonStyle())

Output()

### Compartment equilibria


In [36]:
#############################################################################################################
# Widget interface
output_equilib = widgets.Output()
output_equilib.layout = Layout(border='solid')
output_equilib_save = widgets.Output(Layout=Layout(border='solid'))
output_legend = widgets.Output()
btn_equilib = widgets.Button(description ="View Equilibrium dynamic",button_style='info')
btn_equilib_save = widgets.Button(description = "Save Graph as png",button_style='info')

drp_equilib = widgets.Dropdown(description ="Compartment")
options_arr = []


for i in range(len(comp_arr)):
    options_arr.append(comp_arr[i].name) 
    
drp_equilib.options = options_arr


display(drp_equilib, btn_equilib, output_equilib, output_equilib_save)

#############################################################################################################
# Creating Equilibria figures


def btn_equilib_clicked(b):
    
    #### Instantiating figure
    
    
    with output_equilib_save:
        display(btn_equilib_save)
    
    
    
    #ISOLATING SPECIFIC COMPARTMENT
    temp_name = drp_equilib.value 
    temp_num = 0 
    
    for i in range(len(comp_arr)):
        if comp_arr[i].name == temp_name:
            temp_num = i
            
    shapes_arr =[]
    
    with output_equilib:
        
        output_equilib.clear_output()
        
        f1 = mplt.figure(figsize=(20,10))
        ax = mplt.gca()
        
        ######### Main compartment:
        
        mainx = 32 #starting x-coordinate for main rectangle if not the last compartment
        mainy = 4  #starting y-coordinate for main rectangle
        main_rect = mpatches.Rectangle([mainx,mainy],20,16, fc='#c4c4ff',ec='red')
        main_atpase = mpatches.Ellipse([mainx+3,mainy+16],width=2, height=4,fc = '#964b00')
        main_kcc2 = mpatches.Ellipse([mainx+8,mainy+16],width=4, height=2,fc = '#8f4f66')
        shapes_arr.append(main_rect)
        shapes_arr.append(main_atpase)
        shapes_arr.append(main_kcc2)
        
        m_net_in = 0
        m_net_out = 0

        ######## Main ATPASE:
        
        m_na_atpase = comp_arr[temp_num].d_na_atpase_arr[-1]*1e6
        if m_na_atpase >0: #Sodium coming into the compartment
            m_na_atpase_arrow = mpatches.FancyArrow(mainx+3, mainy+16,0,-2,width=0.2,ec='#e7298a',fc='#e7298a',ls='--')
            ax.text(mainx+2.5,mainy+12.5, str(round(m_na_atpase,1)), fontsize=14)
        else:
            m_na_atpase_arrow = mpatches.FancyArrow(mainx+3, mainy+16,0,+2,width=0.2,ec='#e7298a',fc='#e7298a',ls='--')
            ax.text(mainx+2.5,mainy+19, str(round(m_na_atpase,1)*-1), fontsize=14)
        shapes_arr.append(m_na_atpase_arrow)
        
        m_k_atpase = comp_arr[temp_num].d_k_atpase_arr[-1]*1e6
        if m_k_atpase >0: #Sodium coming into the compartment
            m_k_atpase_arrow = mpatches.FancyArrow(mainx+3, mainy+16,0,-2,width=0.2, ec='#7570b3',fc='#7570b3',ls='--')
            ax.text(mainx+2.5,mainy+12.5, str(round(m_k_atpase,1)), fontsize=14)
        else:
            m_k_atpase_arrow = mpatches.FancyArrow(mainx+3, mainy+16,0,+2,width=0.2, ec='#7570b3',fc='#7570b3',ls='--')
            ax.text(mainx+2.5,mainy+19, str(round(m_k_atpase,1)*-1), fontsize=14)
        shapes_arr.append(m_k_atpase_arrow)
        
        ######## Main KCC2:
        
        m_k_kcc2 = comp_arr[temp_num].d_k_kcc2_arr[-1]*1e6
        if m_k_kcc2 >0: #Potassium entering via KCC2
            m_k_kcc2_arrow = mpatches.FancyArrow(mainx+7, mainy+16,0,-2,width=0.2,ec='#7570b3',fc='#7570b3',ls='--')
            ax.text(mainx+6.5,mainy+12.5, str(round(m_k_kcc2,1)), fontsize=14)
        else:
            m_k_kcc2_arrow = mpatches.FancyArrow(mainx+7, mainy+16,0,+2,width=0.2,ec='#7570b3',fc='#7570b3',ls='--')
            ax.text(mainx+6.5,mainy+19, str(round(m_k_kcc2,1)*-1), fontsize=14)
        shapes_arr.append(m_k_kcc2_arrow)
        
        m_cl_kcc2 = comp_arr[temp_num].d_cl_kcc2_arr[-1]*1e6
        if m_cl_kcc2 >0: #Sodium coming into the compartment
            m_cl_kcc2_arrow = mpatches.FancyArrow(mainx+9, mainy+16,0,-2,width=0.2, ec='#1b9e77',fc='#1b9e77',ls='--')
            ax.text(mainx+8.5,mainy+12.5, str(round(m_cl_kcc2,1)), fontsize=14)
        else:
            m_cl_kcc2_arrow = mpatches.FancyArrow(mainx+9, mainy+16,0,+2,width=0.2, ec='#1b9e77',fc='#1b9e77',ls='--')
            ax.text(mainx+8.5,mainy+19, str(round(m_cl_kcc2,1)*-1), fontsize=14)
        shapes_arr.append(m_cl_kcc2_arrow)
        
            
        ########## Main Na Leak:
        
        m_d_na = comp_arr[temp_num].d_na_leak_arr[-1]*1e6
        if m_d_na >0: #sodium coming into the compartment
            m_na_arrow = mpatches.FancyArrow(mainx+13, mainy+16, 0, -1, width=0.2,ec='#e7298a',fc='#e7298a')
            ax.text(mainx+12.5,mainy+13,str(round(m_d_na,1)), fontsize=14)
            m_net_in += m_d_na #adding sodium coming in
        else:
            m_na_arrow = mpatches.FancyArrow(mainx+13, mainy+16, 0, +1, width=0.2,ec='#e7298a',fc='#e7298a')
            ax.text(mainx+12.5,mainy+18,str(round(m_d_na,1)), fontsize=14)
            m_net_out += -1*m_d_na
        shapes_arr.append(m_na_arrow)

        ########## Main K Leak:
        
        m_d_k = comp_arr[temp_num].d_k_leak_arr[-1]*1e6
        if m_d_k >0: #potassium coming into the compartment
            m_k_arrow = mpatches.FancyArrow(mainx+16, mainy+16, 0, -1, width=0.2,ec='#7570b3',fc='#7570b3')
            ax.text(mainx+15.5,mainy+13,str(round(m_d_k,1)), fontsize=14)
            m_net_in += m_d_k
        else:
            m_k_arrow = mpatches.FancyArrow(mainx+16, mainy+16, 0, +1, width=0.2,ec='#7570b3',fc='#7570b3')
            ax.text(mainx+15.5,mainy+18,str(round(m_d_k,1)), fontsize=14)
            m_net_out += -1*m_d_k
        shapes_arr.append(m_k_arrow)

        ########## Main Cl Leak:
        
        m_d_cl = comp_arr[temp_num].d_cl_leak_arr[-1]*1e6
        if m_d_cl >0: #chloride coming into selected (main) compartment
            m_cl_arrow = mpatches.FancyArrow(mainx+19, mainy+16, 0, -1, width=0.2,ec='#1b9e77',fc='#1b9e77')
            ax.text(mainx+18,mainy+13,str(round(m_d_cl,1)), fontsize=14)
            m_net_in -= m_d_cl #minus chloride coming in
        else:
            m_cl_arrow = mpatches.FancyArrow(mainx+19, mainy+16, 0, +1, width=0.2,ec='#1b9e77',fc='#1b9e77')
            ax.text(mainx+18,mainy+18,str(round(m_d_cl,1)), fontsize=14) 
            m_net_out += m_d_cl
        shapes_arr.append(m_cl_arrow)




        ######## Right Compartment

        rightx = mainx+22
        righty = 4
        right_comp=False
        r_net_in =0
        r_net_out =0

        if temp_num+1 != len(comp_arr):   #if not the last componenent, make rectangle on right

            right_comp = True
            right_rect = mpatches.Rectangle([rightx,righty],20,16, fc='#ebebff')
            right_atpase = mpatches.Ellipse([rightx+3,righty+16],width=2, height=4,fc = '#964b00')
            right_kcc2 = mpatches.Ellipse([rightx+8,righty+16],width=4, height=2,fc = '#8f4f66')
            shapes_arr.append(right_rect)
            shapes_arr.append(right_atpase)
            shapes_arr.append(right_kcc2)
            
            
            #### Right ATPase
            r_na_atpase = comp_arr[temp_num+1].d_na_atpase_arr[-1]*1e6
            if r_na_atpase >0: #Sodium coming into the compartment
                r_na_atpase_arrow = mpatches.FancyArrow(rightx+3, righty+16,0,-2,width=0.2,ec='#e7298a',fc='#e7298a',ls='--')
                ax.text(rightx+2.5,righty+12.5, str(round(r_na_atpase,1)), fontsize=14)
            else:
                r_na_atpase_arrow = mpatches.FancyArrow(rightx+3, righty+16,0,+2,width=0.2,ec='#e7298a',fc='#e7298a',ls='--')
                ax.text(rightx+2.5,righty+19, str(round(r_na_atpase,1)*-1), fontsize=14)
            shapes_arr.append(r_na_atpase_arrow)

            r_k_atpase = comp_arr[temp_num+1].d_k_atpase_arr[-1]*1e6
            if r_k_atpase >0: #Sodium coming into the compartment
                r_k_atpase_arrow = mpatches.FancyArrow(rightx+3, righty+16,0,-2,width=0.2, ec='#7570b3',fc='#7570b3',ls='--')
                ax.text(rightx+2.5,righty+12.5, str(round(r_k_atpase,1)), fontsize=14)
            else:
                r_k_atpase_arrow = mpatches.FancyArrow(rightx+3, righty+16,0,+2,width=0.2, ec='#7570b3',fc='#7570b3',ls='--')
                ax.text(rightx+2.5,righty+19, str(round(r_k_atpase,1)*-1), fontsize=14)
            shapes_arr.append(r_k_atpase_arrow)
            
            ######## Right KCC2:
        
            r_k_kcc2 = comp_arr[temp_num+1].d_k_kcc2_arr[-1]*1e6
            if r_k_kcc2 >0: #Potassium entering via KCC2
                r_k_kcc2_arrow = mpatches.FancyArrow(rightx+7, righty+16,0,-2,width=0.2,ec='#7570b3',fc='#7570b3',ls='--')
                ax.text(rightx+6.5,righty+12.5, str(round(r_k_kcc2,1)), fontsize=14)
            else:
                r_k_kcc2_arrow = mpatches.FancyArrow(rightx+7, righty+16,0,+2,width=0.2,ec='#7570b3',fc='#7570b3',ls='--')
                ax.text(rightx+6.5,righty+19, str(round(r_k_kcc2,1)*-1), fontsize=14)
            shapes_arr.append(r_k_kcc2_arrow)

            r_cl_kcc2 = comp_arr[temp_num+1].d_cl_kcc2_arr[-1]*1e6
            if r_cl_kcc2 >0: #Sodium coming into the compartment
                r_cl_kcc2_arrow = mpatches.FancyArrow(rightx+9, righty+16,0,-2,width=0.2, ec='#1b9e77',fc='#1b9e77',ls='--')
                ax.text(rightx+8.5,righty+12.5, str(round(r_cl_kcc2,1)), fontsize=14)
            else:
                r_cl_kcc2_arrow = mpatches.FancyArrow(rightx+9, righty+16,0,+2,width=0.2, ec='#1b9e77',fc='#1b9e77',ls='--')
                ax.text(rightx+8.5,righty+19, str(round(r_cl_kcc2,1)*-1), fontsize=14)
            shapes_arr.append(r_cl_kcc2_arrow)
        
            
            #d_Na
            r_d_na = comp_arr[temp_num+1].d_na_leak_arr[-1]*1e6
            if r_d_na >0: #sodium coming into the compartment
                r_na_arrow = mpatches.FancyArrow(rightx+13, righty+16, 0, -1, width=0.2,ec='#e7298a',fc='#e7298a')
                ax.text(rightx+12.5,righty+13,str(round(r_d_na,1)), fontsize=14)
                r_net_in +=r_d_na
            else:
                r_na_arrow = mpatches.FancyArrow(rightx+13, righty+16, 0, +1, width=0.2,ec='#e7298a',fc='#e7298a')
                ax.text(rightx+12.5,righty+18,str(round(r_d_na,1)), fontsize=14)
                r_net_out += -1*r_d_na
            shapes_arr.append(r_na_arrow)

            #d_K
            r_d_k = comp_arr[temp_num+1].d_k_leak_arr[-1]*1e6
            if r_d_k >0: #potassium coming into the compartment
                r_k_arrow = mpatches.FancyArrow(rightx+16, righty+16, 0, -1, width=0.2,ec='#7570b3',fc='#7570b3')
                ax.text(rightx+15.5,righty+13,str(round(r_d_k,1)), fontsize=14)
                r_net_in += r_d_k
            else:
                r_k_arrow = mpatches.FancyArrow(rightx+16, righty+16, 0, +1, width=0.2,ec='#7570b3',fc='#7570b3')
                ax.text(rightx+15.5,righty+18,str(round(r_d_k,1)), fontsize=14) 
                r_net_out += -1*r_d_k
            shapes_arr.append(r_k_arrow)


            #d_Cl
            r_d_cl = comp_arr[temp_num+1].d_cl_leak_arr[-1]*1e6
            if r_d_cl >0: #chloride coming into the compartment
                r_cl_arrow = mpatches.FancyArrow(rightx+19, righty+16, 0, -1, width=0.2,ec='#1b9e77',fc='#1b9e77')
                ax.text(rightx+18,righty+13,str(round(r_d_cl,1)), fontsize=14)
                r_net_in -= r_d_cl
            else:
                r_cl_arrow = mpatches.FancyArrow(rightx+19, righty+16, 0, +1, width=0.2,ec='#1b9e77',fc='#1b9e77')
                ax.text(rightx+18,righty+18,str(round(r_d_cl,1)), fontsize=14)  
                r_net_out += r_d_cl
            shapes_arr.append(r_cl_arrow)

            #Right Electrodiffusion

            r_ed_na = ed_arr[temp_num].bound_na_arr[-1]*1e6
            if r_ed_na >0: 
                r_ed_na_arrow = mpatches.FancyArrow(rightx+2, righty+8, -5, 0, width=0.2,ec='#e7298a',fc='#e7298a',ls='dotted')
                ax.text(rightx+2,righty+7.75,str(round(r_ed_na,1)), fontsize=14)
                shapes_arr.append(r_ed_na_arrow)
                r_net_out += r_ed_na
                m_net_in += r_ed_na

            else: #Sodium moving out of cell
                r_ed_na_arrow = mpatches.FancyArrow(rightx-4, righty+8, +5, 0, width=0.2,ec='#e7298a',fc='#e7298a',ls ='dotted')
                ax.text(rightx+2,righty+7.75,str(round(r_ed_na,1)), fontsize=14)
                shapes_arr.append(r_ed_na_arrow)
                r_net_in += -1*r_ed_na
                m_net_out += -1*r_ed_na

            r_ed_k = ed_arr[temp_num].bound_k_arr[-1]*1e6
            if r_ed_k >0: 
                r_ed_k_arrow = mpatches.FancyArrow(rightx+2, righty+6, -5, 0, width=0.2,ec='#7570b3',fc='#7570b3')
                ax.text(rightx-7,righty+5.75,str(round(r_ed_k,1)), fontsize=14)
                shapes_arr.append(r_ed_k_arrow)
                r_net_out += r_ed_k
                m_net_in += r_ed_k
            else:
                r_ed_k_arrow = mpatches.FancyArrow(rightx-4, righty+6, +5, 0, width=0.2,ec='#7570b3',fc='#7570b3')
                ax.text(rightx+2,righty+5.75,str(round(r_ed_k,1)), fontsize=14)
                shapes_arr.append(r_ed_k_arrow)
                r_net_in -= r_ed_k
                m_net_out -= r_ed_k

            r_ed_cl = ed_arr[temp_num].bound_cl_arr[-1]*1e6
            if r_ed_cl >0: 
                r_ed_cl_arrow = mpatches.FancyArrow(rightx+2, righty+4, -5, 0, width=0.2,ec='#1b9e77',fc='#1b9e77')
                ax.text(rightx-7,righty+3.75,str(round(r_ed_cl,1)), fontsize=14)
                shapes_arr.append(r_ed_cl_arrow)
                r_net_out -= r_ed_cl
                m_net_in -= r_ed_cl
            else:
                r_ed_cl_arrow = mpatches.FancyArrow(rightx-4, righty+4, +5, 0, width=0.2,ec='#1b9e77',fc='#1b9e77')
                ax.text(rightx+2,righty+3.75,str(round(r_ed_cl,1)), fontsize=14)
                shapes_arr.append(r_ed_cl_arrow)
                r_net_in += r_ed_cl
                m_net_out += r_ed_cl

            #Right Ion square:
            #r_net_rect = mpatches.Rectangle([rightx+8,righty+7],5,2.5, fc='#ebebff')
            #shapes_arr.append(r_net_rect)
            ax.text(rightx+7.5, righty+8.5,"Net in: " + str(round(r_net_in,2)),fontsize=14) 
            ax.text(rightx+7.5, righty+7.5,"Net out: " + str(round(r_net_out,2)),fontsize=14) 


        ######## Left Compartment

        leftx = mainx-22
        lefty=4
        left_comp=False
        l_net_in =0
        l_net_out =0

        if temp_num+1 != 1:   #if not the first componenent, make rectangle on left

            left_comp = True
            left_rect = mpatches.Rectangle([leftx,lefty],20,16, fc='#ebebff')
            shapes_arr.append(left_rect)
            left_atpase = mpatches.Ellipse([leftx+3,lefty+16],width=2, height=4,fc = '#964b00')
            left_kcc2 = mpatches.Ellipse([leftx+8,lefty+16],width=4, height=2,fc = '#8f4f66')
            shapes_arr.append(left_atpase)
            shapes_arr.append(left_kcc2)
            
            #### Left ATPase
            l_na_atpase = comp_arr[temp_num-1].d_na_atpase_arr[-1]*1e6
            if l_na_atpase >0: #Sodium coming into the compartment
                l_na_atpase_arrow = mpatches.FancyArrow(leftx+3, lefty+16,0,-2,width=0.2,ec='#e7298a',fc='#e7298a',ls='--')
                ax.text(leftx+2.5,lefty+12.5, str(round(l_na_atpase,1)), fontsize=14)
            else:
                l_na_atpase_arrow = mpatches.FancyArrow(leftx+3, lefty+16,0,+2,width=0.2,ec='#e7298a',fc='#e7298a',ls='--')
                ax.text(leftx+2.5,lefty+19, str(round(l_na_atpase,1)*-1), fontsize=14)
            shapes_arr.append(l_na_atpase_arrow)

            l_k_atpase = comp_arr[temp_num-1].d_k_atpase_arr[-1]*1e6
            if l_k_atpase >0: #Sodium coming into the compartment
                l_k_atpase_arrow = mpatches.FancyArrow(leftx+3, lefty+16,0,-2,width=0.2, ec='#7570b3',fc='#7570b3',ls='--')
                ax.text(leftx+2.5,lefty+12.5, str(round(l_k_atpase,1)), fontsize=14)
            else:
                l_k_atpase_arrow = mpatches.FancyArrow(leftx+3, lefty+16,0,+2,width=0.2, ec='#7570b3',fc='#7570b3',ls='--')
                ax.text(leftx+2.5,lefty+19, str(round(l_k_atpase,1)*-1), fontsize=14)
            shapes_arr.append(l_k_atpase_arrow)
            
            ######## Left KCC2:
        
            l_k_kcc2 = comp_arr[temp_num-1].d_k_kcc2_arr[-1]*1e6
            if l_k_kcc2 >0: #Potassium entering via KCC2
                l_k_kcc2_arrow = mpatches.FancyArrow(leftx+7, lefty+16,0,-2,width=0.2,ec='#7570b3',fc='#7570b3',ls='--')
                ax.text(leftx+6.5,lefty+12.5, str(round(l_k_kcc2,1)), fontsize=14)
            else:
                l_k_kcc2_arrow = mpatches.FancyArrow(leftx+7, lefty+16,0,+2,width=0.2,ec='#7570b3',fc='#7570b3',ls='--')
                ax.text(leftx+6.5,lefty+19, str(round(l_k_kcc2,1)*-1), fontsize=14)
            shapes_arr.append(l_k_kcc2_arrow)

            l_cl_kcc2 = comp_arr[temp_num-1].d_cl_kcc2_arr[-1]*1e6
            if l_cl_kcc2 >0: #Sodium coming into the compartment
                l_cl_kcc2_arrow = mpatches.FancyArrow(leftx+9, lefty+16,0,-2,width=0.2, ec='#1b9e77',fc='#1b9e77',ls='--')
                ax.text(leftx+8.5,lefty+12.5, str(round(l_cl_kcc2,1)), fontsize=14)
            else:
                l_cl_kcc2_arrow = mpatches.FancyArrow(leftx+9, lefty+16,0,+2,width=0.2, ec='#1b9e77',fc='#1b9e77',ls='--')
                ax.text(leftx+8.5,lefty+19, str(round(l_cl_kcc2,1)*-1), fontsize=14)
            shapes_arr.append(l_cl_kcc2_arrow)
            
            #d_Na
            l_d_na = comp_arr[temp_num-1].d_na_leak_arr[-1]*1e6
            if l_d_na >0: #sodium coming into the left compartment
                l_na_arrow = mpatches.FancyArrow(leftx+13, lefty+16, 0, -1, width=0.2, ec='#e7298a',fc='#e7298a')
                ax.text(leftx+12.5,lefty+13,str(round(l_d_na,1)), fontsize=14)
                l_net_in += l_d_na
            else:
                l_na_arrow = mpatches.FancyArrow(leftx+13, lefty+16, 0, +1, width=0.2,ec='#e7298a',fc='#e7298a')
                ax.text(leftx+12.5,lefty+18,str(round(l_d_na,1)), fontsize=14)
                l_net_out += -1*l_d_na
            shapes_arr.append(l_na_arrow)

            #d_K
            l_d_k = comp_arr[temp_num-1].d_k_leak_arr[-1]*1e6
            if l_d_k >0: #potassium coming into the compartment
                l_k_arrow = mpatches.FancyArrow(leftx+16, lefty+16, 0, -1, width=0.2,ec='#7570b3',fc='#7570b3')
                ax.text(leftx+15.5,lefty+13,str(round(l_d_k,1)), fontsize=14)
                l_net_in += l_d_k
            else:
                l_k_arrow = mpatches.FancyArrow(leftx+16, lefty+16, 0, +1, width=0.2,ec='#7570b3',fc='#7570b3')
                ax.text(leftx+15.5,lefty+18,str(round(l_d_k,1)), fontsize=14)  
                l_net_out += -1*l_d_k
            shapes_arr.append(l_k_arrow)


            #d_Cl
            l_d_cl = comp_arr[temp_num-1].d_cl_leak_arr[-1]*1e6
            if l_d_cl >0: #chloride coming into the compartment
                l_cl_arrow = mpatches.FancyArrow(leftx+19, lefty+16, 0, -1, width=0.2,ec='#1b9e77',fc='#1b9e77')
                ax.text(leftx+18,lefty+13,str(round(l_d_cl,1)), fontsize=14)
                l_net_in += -1*l_d_cl
            else:
                l_cl_arrow = mpatches.FancyArrow(leftx+19, lefty+16, 0, +1, width=0.2,ec='#1b9e77',fc='#1b9e77')
                ax.text(leftx+18,lefty+18,str(round(l_d_cl,1)), fontsize=14) 
                l_net_out += l_d_cl
            shapes_arr.append(l_cl_arrow)


            #Left compartment boundary (electrodiffusion):

            l_ed_na = ed_arr[temp_num-1].bound_na_arr[-1]*1e6
            if l_ed_na >0: #sodium leaving the L compartment, entering main compartment via ED
                l_ed_na_arrow = mpatches.FancyArrow(mainx+2, mainy+8, -5, 0, width=0.2,ec='#e7298a',fc='#e7298a',hatch ='+')
                ax.text(mainx-7,mainy+7.75,str(round(l_ed_na,2)), fontsize=14)
                shapes_arr.append(l_ed_na_arrow)
                l_net_in += l_ed_na
                m_net_out += l_ed_na
            else: # sodium entering via ED
                l_ed_na_arrow = mpatches.FancyArrow(mainx-4, mainy+8, +5, 0, width=0.2,ec='#e7298a',fc='#e7298a',ls =':')
                ax.text(mainx+2,mainy+7.75,str(round(l_ed_na,2)), fontsize=14)
                shapes_arr.append(l_ed_na_arrow)
                l_net_out -= l_ed_na
                m_net_in -= l_ed_na

            l_ed_k = ed_arr[temp_num-1].bound_k_arr[-1]*1e6
            if l_ed_k >0: #K leaving the compartment via ED
                l_ed_k_arrow = mpatches.FancyArrow(mainx+2, mainy+6, -5, 0, width=0.2,ec='#7570b3',fc='#7570b3')
                ax.text(mainx-7,mainy+5.75,str(round(l_ed_k,2)), fontsize=14)
                shapes_arr.append(l_ed_k_arrow)
                l_net_in += l_ed_k
                m_net_out += l_ed_k
            else:
                l_ed_k_arrow = mpatches.FancyArrow(mainx-4, mainy+6, +5, 0, width=0.2,ec='#7570b3',fc='#7570b3')
                ax.text(mainx+2,mainy+5.75,str(round(l_ed_k,2)), fontsize=14)
                shapes_arr.append(l_ed_k_arrow)
                l_net_out -= l_ed_k
                m_net_in -= l_ed_k

            l_ed_cl = ed_arr[temp_num-1].bound_cl_arr[-1]*1e6
            if l_ed_cl >0: #sodium coming into the compartment
                l_ed_cl_arrow = mpatches.FancyArrow(mainx+2, mainy+4, -5, 0, width=0.2,ec='#1b9e77',fc='#1b9e77')
                ax.text(mainx-7,mainy+3.75,str(round(l_ed_cl,2)), fontsize=14)
                shapes_arr.append(l_ed_cl_arrow)
                l_net_in -= l_ed_cl
                m_net_out -= l_ed_cl
            else:
                l_ed_cl_arrow = mpatches.FancyArrow(mainx-4, mainy+4, +5, 0, width=0.2,ec='#1b9e77',fc='#1b9e77')
                ax.text(mainx+2,mainy+3.75,str(round(l_ed_cl,2)), fontsize=14)
                shapes_arr.append(l_ed_cl_arrow)
                l_net_out += l_ed_cl
                m_net_in += l_ed_cl



            #Left Ion square:
            #l_net_rect = mpatches.Rectangle([leftx+8,lefty+7],5,2.5, fc='#ebebff')
            #shapes_arr.append(l_net_rect)
            ax.text(leftx+7.5, lefty+8.5,"Net in: " + str(round(l_net_in,2)),fontsize=14) 
            ax.text(leftx+7.5, lefty+7.5,"Net out: " + str(round(l_net_out,2)),fontsize=14) 

        #Net Ion square:
        #m_net_rect = mpatches.Rectangle([mainx+8,mainy+7],5,2.5,fc='#ebebff')
        #shapes_arr.append(m_net_rect)
        ax.text(mainx+7.5, mainy+8.5,"Net in: " + str(round(m_net_in,2)),fontsize=14) 
        ax.text(mainx+7.5, mainy+7.5,"Net out: " + str(round(m_net_out,2)),fontsize=14) 



        ######### legend
        leg_rect = mpatches.Rectangle([70,20],8,10, fc='white', ec='black')
        shapes_arr.append(leg_rect)
        ax.text(70.5,28.5,"Legend", fontsize=20)
        leg_na_arrow = mpatches.FancyArrow(71, 27, 2, 0, width=0.1,ec='#e7298a',fc='#e7298a')
        shapes_arr.append(leg_na_arrow)
        ax.text(74,26.5,"Na+", fontsize=14)
        leg_k_arrow = mpatches.FancyArrow(71, 25, 2, 0, width=0.1,ec='#7570b3',fc='#7570b3')
        shapes_arr.append(leg_k_arrow)
        ax.text(74,24.5,"K+", fontsize=14)
        leg_cl_arrow = mpatches.FancyArrow(71, 23, 2, 0, width=0.1,ec='#1b9e77',fc='#1b9e77')
        shapes_arr.append(leg_cl_arrow)
        ax.text(74,22.5,"Cl-", fontsize=14)






        for j in range(len(shapes_arr)):
            ax.add_patch(shapes_arr[j])

        

        ax.text(mainx+8.5,mainy+1,temp_name, fontsize=16)

        if left_comp == True: 
            ax.text(leftx+8.5,lefty+1,comp_arr[temp_num-1].name, fontsize=16)

        if right_comp == True: 
            ax.text(rightx+8.5,righty+1,comp_arr[temp_num+1].name, fontsize=16)

        mplt.axis("Scaled")
        #ax.text(mainx,30,"**All values are in 10^6 mM", fontsize=16)
        ax.axis("off")
        
        mplt.show()
        
        def btn_equilib_save_clicked(b):
    
            f1.savefig("BOUNDARY",format='png')
            print("Figure titled BOUNDARY saved in local directory")
                       
        btn_equilib_save.on_click(btn_equilib_save_clicked)


        
###########################################################################################################        
        

        
        

#############################################################################################################    
# Connecting button to backend

btn_equilib.on_click(btn_equilib_clicked)


Dropdown(description='Compartment', options=('Comp1', 'Comp2', 'Comp3'), value='Comp1')

Button(button_style='info', description='View Equilibrium dynamic', style=ButtonStyle())

Output(layout=Layout(border='solid'))

Output()

## D) Generate Simulation Report

In [15]:
##################################
# WIDGET INITIALIZATION:
##################################

btn_report = widgets.Button(description = "Generate Report", button_style = 'info' )
output_report = widgets.Output()

display(btn_report, output_report)




df_start = pd.DataFrame()
#df_start.index = ['Radius', 'Length', 'Volume', 'Na_i', 'K_i', 'Cl_i', 'X_i', 'z_i', 'ATP pump rate', 'KCC2 pump rate',
 #               'Vm', 'Ek', 'ECl']


def btn_report_clicked(b):
    output_report.clear_output
    print(df_start)
    
btn_report.on_click(btn_report_clicked)

Button(button_style='info', description='Generate Report', style=ButtonStyle())

Output()

In [16]:
print(comp_arr[0].d_na_atpase)

-1.1595575664051325e-07
